In [ ]:
!pip install git+https://github.com/openai/whisper.git -q
!pip install -q -U google-genai
!pip install gTTS

In [ ]:
import whisper
import os
from gtts import gTTS

In [ ]:
from google import genai
import os

os.environ['GEMINI_API_KEY'] = ' ' #inserir chave de api aqui
client = genai.Client()

In [ ]:
model = whisper.load_model("small")
language = 'pt'
time = 5 # tempo que o microfone estará ligado em segundos

In [ ]:
# @title
# Referência: https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be

from IPython.display import Javascript, Audio
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})

var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=time):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.wav','wb') as f:
    f.write(b)
  return 'audio.wav'  # or webm

In [ ]:
print('Ouvindo...\n')
record_file = record()

print('Processando...\n')
result = model.transcribe(record_file, fp16=False, language=language)
transcription = result['text']
print(f'Pronto! Você disse: \'{transcription}\'')

response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents=transcription
)

print('Resposta: ')
gemini_response = response.text
print(gemini_response)

gtts_object = gTTS(text=gemini_response, lang=language, slow=False)
response_audio = "/content/response_audio.wav"
gtts_object.save(response_audio)

display(Audio(response_audio, autoplay=True))